In [149]:
import pandas as pd
from datetime import datetime
import json

In [150]:
df = pd.read_csv('../data/raw/2020/dados/microdados_ed_basica_2020.csv', delimiter=";", encoding="latin1", low_memory=False)

In [151]:
{column[:3] for column in df.columns}

{'CO_', 'DS_', 'DT_', 'IN_', 'NO_', 'NU_', 'QT_', 'SG_', 'TP_'}

In [152]:
columns_to_drop = ["CO_REGIAO", "CO_UF", 'CO_MUNICIPIO', 'CO_MESORREGIAO', 'CO_MICRORREGIAO']

In [153]:
df.drop(columns=columns_to_drop, inplace=True)

In [154]:
boolean_columns = [column for column in df.columns if column.startswith("IN")]
integer_columns = [column for column in df.columns
                       if column.startswith("QT")]
date_columns = [column for column in df.columns if column.startswith("DT")]
categorical_columns = [column for column in df.columns if column.startswith("TP")] + [
    'CO_LINGUA_INDIGENA_1', 'CO_LINGUA_INDIGENA_2', 'CO_LINGUA_INDIGENA_3'
]
identifier_columns = ['NU_DDD', 'NU_TELEFONE', 'NU_CNPJ_ESCOLA_PRIVADA',
       'NU_CNPJ_MANTENEDORA', 'CO_ESCOLA_SEDE_VINCULADA', 'CO_IES_OFERTANTE', 
                      'CO_DISTRITO', 'CO_ENTIDADE', 'CO_CEP']

In [155]:
df[boolean_columns] = df[boolean_columns].replace(to_replace=9.0, value=None).astype("bool")

In [156]:
df[integer_columns] = df[integer_columns].astype("Int32")

In [157]:
for column in date_columns:
    df[column] = df[column].apply(
        lambda date: 
            datetime.strptime(date, "%d%b%Y:%H:%M:%S") 
               if isinstance(date, str) 
            else None
    )

In [158]:
for value in df["DT_ANO_LETIVO_INICIO"].values:
    try:
        if isinstance(value, str):
            datetime.strptime(value, "%d%b%Y:%H:%M:%S")
    except Exception as e:
        print(value)
        raise e

In [159]:
with open("map_categorical_columns.json") as file:
    map_categorical_columns = json.load(file)

In [177]:
if columns_not_mapped := set(categorical_columns).difference(
        map_categorical_columns.keys()):
    raise Exception()

In [161]:
df[categorical_columns] = df[categorical_columns].astype("float").fillna(-1).astype("int").astype("str")
for column in categorical_columns:
    if column in map_categorical_columns.keys():
        df[column] = df[column].replace(map_categorical_columns[column])
    else:
        print(f"{column} not mapped")
df[categorical_columns] = df[categorical_columns].replace(["-1", "9"], None)

In [162]:
for column in categorical_columns:
    print(f"{column}: {df[column].unique()}\n")

TP_DEPENDENCIA: ['Estadual' 'Municipal' 'Privada' 'Federal']

TP_CATEGORIA_ESCOLA_PRIVADA: [None 'Filantrópica' 'Particular' 'Confessional' 'Comunitária']

TP_LOCALIZACAO: ['Rural' 'Urbana']

TP_LOCALIZACAO_DIFERENCIADA: ['Terra indígena' 'A escola não está em área de localização diferenciada'
 None 'Área de assentamento'
 'Área onde se localiza comunidade remanescente de quilombos']

TP_SITUACAO_FUNCIONAMENTO: ['Em Atividade' 'Extinta (ano do censo)' 'Paralisada']

TP_CONVENIO_PODER_PUBLICO: [None 'Estadual' 'Estadual e Municipal' 'Municipal']

TP_REGULAMENTACAO: ['Em tramitação' 'Sim' None 'Não']

TP_RESPONSAVEL_REGULAMENTACAO: ['Estadual' 'Municipal' None 'Federal' 'Estadual e Municipal'
 'Federal e Estadual']

TP_OCUPACAO_PREDIO_ESCOLAR: ['Próprio' None 'Alugado' 'Cedido']

TP_OCUPACAO_GALPAO: [None 'Próprio' 'Cedido' 'Alugado']

TP_REDE_LOCAL: [None 'A cabo' 'Não há rede local interligando computadores'
 'A cabo e Wireless' 'Wireless']

TP_INDIGENA_LINGUA: ['Em Língua Indígena e e

In [163]:
df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype("int16")

In [164]:
df[identifier_columns] = df[identifier_columns].astype("str")

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224229 entries, 0 to 224228
Columns: 365 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: Int32(113), bool(210), datetime64[ns](2), int16(1), object(39)
memory usage: 236.3+ MB


In [166]:
df.select_dtypes("int32").columns

Index(['QT_SALAS_EXISTENTES', 'QT_SALAS_UTILIZADAS_DENTRO',
       'QT_SALAS_UTILIZADAS_FORA', 'QT_SALAS_UTILIZADAS',
       'QT_SALAS_UTILIZA_CLIMATIZADAS', 'QT_SALAS_UTILIZADAS_ACESSIVEIS',
       'QT_EQUIP_DVD', 'QT_EQUIP_SOM', 'QT_EQUIP_TV', 'QT_EQUIP_LOUSA_DIGITAL',
       ...
       'QT_TUR_FUND_AF', 'QT_TUR_MED', 'QT_TUR_PROF', 'QT_TUR_PROF_TEC',
       'QT_TUR_EJA', 'QT_TUR_EJA_FUND', 'QT_TUR_EJA_MED', 'QT_TUR_ESP',
       'QT_TUR_ESP_CC', 'QT_TUR_ESP_CE'],
      dtype='object', length=113)

In [167]:
df.select_dtypes("bool").columns

Index(['IN_VINCULO_SECRETARIA_EDUCACAO', 'IN_VINCULO_SEGURANCA_PUBLICA',
       'IN_VINCULO_SECRETARIA_SAUDE', 'IN_VINCULO_OUTRO_ORGAO',
       'IN_CONVENIADA_PP', 'IN_MANT_ESCOLA_PRIVADA_EMP',
       'IN_MANT_ESCOLA_PRIVADA_ONG', 'IN_MANT_ESCOLA_PRIVADA_OSCIP',
       'IN_MANT_ESCOLA_PRIV_ONG_OSCIP', 'IN_MANT_ESCOLA_PRIVADA_SIND',
       ...
       'IN_FUND_AF', 'IN_MED', 'IN_PROF', 'IN_PROF_TEC', 'IN_EJA',
       'IN_EJA_FUND', 'IN_EJA_MED', 'IN_ESP', 'IN_ESP_CC', 'IN_ESP_CE'],
      dtype='object', length=210)

In [168]:
df.select_dtypes("object").columns

Index(['NO_REGIAO', 'NO_UF', 'SG_UF', 'NO_MUNICIPIO', 'NO_MESORREGIAO',
       'NO_MICRORREGIAO', 'CO_DISTRITO', 'CO_ENTIDADE', 'NO_ENTIDADE',
       'TP_DEPENDENCIA', 'TP_CATEGORIA_ESCOLA_PRIVADA', 'TP_LOCALIZACAO',
       'TP_LOCALIZACAO_DIFERENCIADA', 'DS_ENDERECO', 'NU_ENDERECO',
       'DS_COMPLEMENTO', 'NO_BAIRRO', 'CO_CEP', 'NU_DDD', 'NU_TELEFONE',
       'TP_SITUACAO_FUNCIONAMENTO', 'CO_ORGAO_REGIONAL',
       'TP_CONVENIO_PODER_PUBLICO', 'NU_CNPJ_ESCOLA_PRIVADA',
       'NU_CNPJ_MANTENEDORA', 'TP_REGULAMENTACAO',
       'TP_RESPONSAVEL_REGULAMENTACAO', 'CO_ESCOLA_SEDE_VINCULADA',
       'CO_IES_OFERTANTE', 'TP_OCUPACAO_PREDIO_ESCOLAR', 'TP_OCUPACAO_GALPAO',
       'TP_REDE_LOCAL', 'TP_INDIGENA_LINGUA', 'CO_LINGUA_INDIGENA_1',
       'CO_LINGUA_INDIGENA_2', 'CO_LINGUA_INDIGENA_3',
       'TP_PROPOSTA_PEDAGOGICA', 'TP_AEE', 'TP_ATIVIDADE_COMPLEMENTAR'],
      dtype='object')

In [169]:
df.to_parquet("../data/transformed", engine='pyarrow', compression='snappy', index=None, partition_cols=["NU_ANO_CENSO"])